# Scraper de propiedades de Argenprop

Este notebook se utiliza para scrapear las propiedades en alquiler disponibles dentro de la página web Argenprop.

## Extracción de datos

Importamos las librerías

In [ ]:
import requests
import urllib
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

Método get_page. Forma la URL y hace un GET a Argenprop.

In [ ]:
def get_page(endpoint, params = None):
    BASE_URL = "https://www.argenprop.com/"
    parameters = []

    if params is not None:
        page = params.get('page')
        if page:
            parameters.append(f"pagina-{page}")
    
        sort = params.get('sort')
        if sort:
            parameters.append(f"orden-{sort}")

    url = f'{BASE_URL}{endpoint}?{"&".join(parameters)}'
    
    print(f"[{get_page.__name__}] Obteniendo página: {url}")
    return requests.get(url).content

El método get_property_details obtiene los detalles de la propiedad.

In [ ]:
def get_property_details(url):
    response = get_page(url)
    soup = BeautifulSoup(response, "html.parser")
    to_return = {
        'address': None,
        'floor': None,
        'neighborhood': None,
        'city': None,
        'expenses': None,
        'price': None,
        'sup_cubierta': None,
        'sup_total': None
    }

    # Extracting basic location information safely
    titlebar_address = soup.find('h2', class_='titlebar__address')
    titlebar_title = soup.find('h2', class_='titlebar__title')

    if titlebar_address:
        address_parts = titlebar_address.get_text(strip=True).split(',')
        to_return['address'] = address_parts[0].strip()
        to_return['floor'] = address_parts[1].strip() if len(address_parts) > 1 else None

    if titlebar_title:
        title_parts = titlebar_title.get_text(strip=True).replace('Alquiler en', '').split(',')
        to_return['neighborhood'] = title_parts[0].strip() if len(title_parts) > 0 else None
        to_return['city'] = title_parts[1].strip() if len(title_parts) > 1 else None

    # Extracting price and expenses
    price_tag = soup.find(class_='titlebar__price')
    expenses_tag = soup.find(class_='titlebar__expenses')

    if price_tag:
        price_text = price_tag.get_text(strip=True).replace('$', '').replace('.', '').strip()
        try:
            to_return['price'] = int(price_text)
        except ValueError:
            to_return['price'] = None

    if expenses_tag:
        expenses_text = expenses_tag.get_text(strip=True).replace('+', '').replace('$', '').replace('expensas', '').replace('.', '').strip()
        try:
            to_return['expenses'] = int(expenses_text)
        except ValueError:
            to_return['expenses'] = None

    # Extracting detailed property features
    feature_sections = soup.find_all('ul', class_='property-features')

    for ul in feature_sections:
        for li in ul.find_all('li'):
            p_tag = li.find('p')
            strong_tag = li.find('strong')

            if not p_tag or not strong_tag:
                continue

            key = p_tag.get_text(strip=True).split(':')[0].strip().lower().replace(' ', '_').replace('.', '')
            value = strong_tag.get_text(strip=True).replace(' m2', '').replace(',', '.').replace('$', '').replace('USD', '').strip()

            try:
                value = int(value)
            except ValueError:
                try:
                    value = float(value)
                except ValueError:
                    value = value

            to_return[key] = value

    return to_return


process_property procesa cada propiedad del listado.

In [ ]:
def process_property(item):
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    url = item.find('a')['href'][1:] if item.find('a') else None # Tomamos a partir del 2do caracter para evitar el /.
    advertiser_id = item.find('span', {'data-anunciante-id': True}).get('data-anunciante-id') if item.find('span', {'data-anunciante-id': True}) else None
    currency = item.find('span', {'data-moneda': True}).get('data-moneda') if item.find('span', {'data-moneda': True}) else None
    price = item.find('span', {'data-precio': True}).get('data-precio') if item.find('span', {'data-precio': True}) else None
        
    details = get_property_details(url)

    # Guardamos la propiedad en el objeto properties.
    return {
        'property_url': url,
        'advertiser_id': advertiser_id,
        'currency': currency,
        'price': price,
        'date_read': timestamp,
        **details
    }

Declaramos los filtros que se pueden mandar en la URL

In [ ]:
#TODO: Eliminar filtro por rooms

listing_type = "casas-o-departamentos-o-ph"
rent_type = "alquiler"
rooms = "1-dormitorio-o-2-dormitorios-o-3-dormitorios-o-4-dormitorios"

#neighborhoods = "capital-federal"

neighborhoods = "agronomia-o-br-santa-rita-o-caballito-o-chacarita-o-colegiales-o-flores-o-parque-centenario-o-parque-chacabuco-o-paternal-o-velez-sarsfield-o-villa-crespo-o-villa-del-parque-o-villa-devoto-o-villa-general-mitre-o-villa-urquiza"

Definimos el endpoint y los query params iniciales. Nos traemos la primera página del resultado e iteramos por cada página con propiedades.

In [ ]:
endpoint = f'{listing_type}/{rent_type}/{neighborhoods}/{rooms}'
params = {
    "page": 1,
    "sort": "menorprecio"
}

page = get_page(endpoint, params)
soup = BeautifulSoup(page, "html.parser")
listing_items = soup.find_all(class_='listing__item')

properties = []

# Iteramos hasta que el botón de siguiente página este inactivo.
while not "pagination__page--disable" in soup.find(class_=['pagination__page-next']).get('class', []):
    
    # Cada propiedad hace un GET al /show de la misma. 
    # Agregamos hilos para mejorar el rendimiento.
    with ThreadPoolExecutor(max_workers=20) as executor:
        results = executor.map(process_property, listing_items)
    
    properties.extend(results)

    params["page"] += 1
    page = get_page(endpoint, params)
    soup = BeautifulSoup(page, "html.parser")
    print(f'Procesando página: {params["page"]}')
    
    listing_items = soup.find_all(class_='listing__item')

In [ ]:
# Convertimos las propiedades a un df.
df = pd.DataFrame(properties)

# Guardamos el dataframe en un csv.
csv_path = f"./{datetime.now().strftime('%Y%m%d_%H%M%S')}_propiedades.csv"
df.to_csv(csv_path, index=False)
    
print(f"Se guardó el dataframe en: {csv_path}.")